Requirement: local python installation on a Windows machine, with access to a chromedriver with GUI

In [1]:
# !pip install beautifulsoup4


In [2]:
# !pip install selenium


In [3]:
# !pip install pywin32


In [32]:
import pandas as pd, time, requests, json
import urllib.parse
import urllib
from bs4 import BeautifulSoup
import win32clipboard
import numpy as np


In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

# make sure the path is correct for your chromedriver. can place next to it
# driver = webdriver.Chrome('./chromedriver.exe')
# driver = webdriver.Chrome('C:/Program Files/chromedriver/chromedriver.exe')
driver = webdriver.Edge("C:/Program Files/chromedriver/msedgedriver.exe")
# opens Chrome, maximize window
driver.maximize_window()


C:\Users\csala\AppData\Local\Temp\ipykernel_13216\1784798272.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge("C:/Program Files/chromedriver/msedgedriver.exe")


# Stage 0
Get keywords, construct mining chains

In [137]:
mi = pd.read_excel("mi_clean.xlsx").set_index('Contact Id')
imgs=json.load(open('imgs.json','r'))
len(mi),len(imgs)


(1096, 737)

In [66]:
# !pip install opencv-python

In [67]:
import cv2
k=300
x=1770
y=872

# Stage 1
Mine profile links

In [68]:
driver.get("https://www.linkedin.com/")
# sign in
actions = ActionChains(driver)


In [95]:
import os
from os.path import isfile, join
import shutil
import urllib.request
mypath='img/'
onlyimgs = [f[:-4] for f in os.listdir(mypath) if isfile(join(mypath, f))]

In [73]:
for i in list(imgs.keys())[:]:
  f=str(mi.loc[i,'public_identifier'])
  if f!='nan':
    if f not in onlyimgs:
      driver.get(imgs[i])
      driver.save_screenshot('img/'+f+".png")
      time.sleep(0.1+np.random.uniform(5)/10.0) #szunet random ideig oldalak kozott
      img = cv2.imread('img/'+f+".png")
      cropImg = img[y:y+k, x:x+k]
      cv2.imwrite('img/'+f+".png",cropImg)
      time.sleep(0.1+np.random.uniform(5)/10.0) #szunet random ideig oldalak kozott
  

In [75]:
cv2.destroyAllWindows()

In [89]:
framed={i:imgs[i] for i in imgs if 'framed' in imgs[i]}
open('framed.json','w').write(json.dumps(framed))

2814

In [102]:
for i in framed:
  f=str(mi.loc[i,'public_identifier'])
  print(i,f,mi.loc[i,'link'])
  if f not in onlyimgs:
    driver.get(imgs[i])
    driver.save_screenshot('img/'+f+".png")
    time.sleep(0.1+np.random.uniform(5)/10.0) #szunet random ideig oldalak kozott
    img = cv2.imread('img/'+f+".png")
    cropImg = img[y:y+k, x:x+k]
    cv2.imwrite('img/'+f+".png",cropImg)
    time.sleep(0.1+np.random.uniform(5)/10.0) #szunet random ideig oldalak kozott
  

0031v00002Y10flAAB nan nan
0031v00002Z9tDqAAJ dora-szendeczki https://www.linkedin.com/in/dora-szendeczki?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACPmmfEBhj7ExTMsjfwzesQYAE5vJPCWkuc
0030Y000013H4P4QAK zita-mosolygo-5426b819b https://www.linkedin.com/in/zita-mosolygo-5426b819b?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC7pbAIBa-y9CBkhSM1MT9N_21ic9EkotYU
0030Y000014yYf6QAE l%C3%A1szl%C3%B3-m%C3%A1t%C3%A9-476331237 https://www.linkedin.com/in/l%C3%A1szl%C3%B3-m%C3%A1t%C3%A9-476331237?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADr1kFoBum2m7xUjcSGV4vg33HtJsILh7rw
0030Y000014yYfEQAU hanna-p%C3%A1lya-b7b93713b https://www.linkedin.com/in/hanna-p%C3%A1lya-b7b93713b?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACIhxM8Bgqlc6pHt6-biOmUM5cI-7HYjF40
0030Y000015aGejQAE miksa-kasza-%C3%A1rp%C3%A1si-5b50181a2 https://www.linkedin.com/in/miksa-kasza-%C3%A1rp%C3%A1si-5b50181a2?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC9YKlkBPbHtUi5ThHzHq1subGcZQOJZMPs
0030Y000018pSOjQAM gretamil

Manual fixes, then

In [103]:
onlyimgs = [f[:-4] for f in os.listdir(mypath) if isfile(join(mypath, f))]

In [109]:
di=pd.DataFrame(onlyimgs)
di['Image']=True


In [127]:
dr=pd.DataFrame(framed,index=['0']).T
dr['0']=True
dr=dr.rename(columns={'0':'Open to opportunities'})

In [141]:
dm=pd.DataFrame(imgs,index=['0']).T
dm=dm.rename(columns={'0':'Image link'})

In [143]:
mi=mi.join(di.set_index(0),on='public_identifier').join(dr).join(dm).drop('Unnamed: 0',axis=1)

In [144]:
mi.to_excel('mi_full.xlsx')

Save rounds

In [35]:
open('second_round.json','w').write(json.dumps([i for i in cards if cards[i]]))

3410

In [36]:
open('first_round.json','w').write(json.dumps(\
  [{links_full[i][0].split('/in/')[1].split('?')[0]:i for i in links_full}[j] for j in nodes]\
  ))

15356

In [37]:
first_round=json.load(open('first_round.json','r'))
second_round=json.load(open('second_round.json','r'))

In [38]:
[i for i in first_round if i in second_round]

[]

In [39]:
len(first_round),len(second_round)

(698, 155)

Extract links but manual check second rounds if plausible